# <center> **Generación de tablas en MySQL**

In [ ]:
!pip install mysql.connector

In [ ]:
import pandas as pd
import numpy as np
import requests
from sqlalchemy import create_engine
import sys
import pymysql
import mysql
import mysql.connector

Para ejecutar el siguiente script es necesario ejecutar lo siguiente:

- [mysql installer community](https://dev.mysql.com/downloads/installer/) : Instala motor de base de datos
- [mysql_conector](https://dev.mysql.com/doc/connector-python/en/connector-python-installation.html) : genera conección de python con MySQL

En MySQL se deberá crear primero la base (create new schema) con la cual se hará la conexión: 

##  Conección 

In [19]:
# Configuración para conexión
user='root'
password='astro123'
host='localhost'
database='sesna'

In [20]:
# coneccion
db= mysql.connector.connect(user=user, 
                            password=password,
                            host=host,
                            database=database)
#cnx.close()

In [21]:
cursor = db.cursor()


### Generación de tabla

En las siguinetes líneas se detalla la creación de los campos (tipo de datos) de la tabla inegi, en la base SEANA. 


In [12]:

cursor.execute("""create TABLE inegi(
    id integer PRIMARY KEY,
    CVE_ENT numeric,
    NOM_ENT text,
    NOM_ABR text,
    CVE_MUN numeric,
    NOM_MUN	text,
    CVE_LOC	numeric,
    NOM_LOC	text,
    AMBITO	text,
    LATITUD	text,
    LONGITUD text,	
    LAT_DECIMAL	numeric,
    LON_DECIMAL	numeric,
    ALTITUD	numeric,
    CVE_CARTA text,	
    POB_TOTAL text,
    POB_MASCULINA text,	    
    POB_FEMENINA text,	
    TOTAL_VIVIENDAS_HABITADAS text
)
""")
db.commit()

Una vez creada la tabla, procesamos los datos para subirlos a la tabla creada.

In [5]:
# lectura de base INEGI para carga a base de datos
base = pd.read_excel('C:/Users/jcmartinez/Desktop/Presentación Python/datasets/AGEEML_2023311323291.xlsx')
# procesamiento
base['CVE_ENT'] = base['CVE_ENT'].astype('int')
base['CVE_MUN'] = base['CVE_MUN'].astype('int')
base['CVE_LOC'] = base['CVE_LOC'].astype('int')
base['LAT_DECIMAL'] = base['LAT_DECIMAL'].astype('float')
base['LON_DECIMAL'] = base['LON_DECIMAL'].astype('float')
base['POB_TOTAL'] = base['POB_TOTAL'].replace('*',np.nan).replace('-',np.nan).astype('str')
base['POB_MASCULINA'] = base['POB_MASCULINA'].replace('*',np.nan).replace('-',np.nan).astype('str')
base['POB_FEMENINA'] = base['POB_FEMENINA'].replace('*',np.nan).replace('-',np.nan).astype('str')
base['TOTAL.DE.VIVIENDAS.HABITADAS'] = base['TOTAL.DE.VIVIENDAS.HABITADAS'].replace('*',np.nan).replace('-',np.nan).astype('str')
base.head()


,MAPA,Estatus,CVE_ENT,NOM_ENT,NOM_ABR,CVE_MUN,NOM_MUN,CVE_LOC,NOM_LOC,AMBITO,LATITUD,LONGITUD,LAT_DECIMAL,LON_DECIMAL,ALTITUD,CVE_CARTA,POB_TOTAL,POB_MASCULINA,POB_FEMENINA,TOTAL.DE.VIVIENDAS.HABITADAS
0,10010001,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,1,Aguascalientes,U,"21°52´47.362N""","102°17´45.768W""",21.88,-102.30,1878,F13D19,863893,419168,444725,246259
1,10010094,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,94,Granja Adelita,R,"21°52´18.749N""","102°22´24.710W""",21.87,-102.37,1901,F13D18,5,nan,nan,2
2,10010096,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,96,Agua Azul,R,"21°53´01.522N""","102°21´25.639W""",21.88,-102.36,1861,F13D18,41,24,17,12
3,10010100,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,100,Rancho Alegre,R,"21°51´16.556N""","102°22´21.884W""",21.85,-102.37,1879,F13D18,0,0,0,0
4,10010102,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,102,Los Arbolitos [Rancho],R,"21°46´48.650N""","102°21´26.261W""",21.78,-102.36,1861,F13D18,8,nan,nan,2



El siguiente paso consite en subir los datos a la tabla de inegi ya generada.

In [13]:
# Agrega registro a la base en PostgreSQL
#cursor = db.cursor()
query = """INSERT INTO inegi (id, CVE_ENT,NOM_ENT,NOM_ABR,CVE_MUN,NOM_MUN,CVE_LOC,
    NOM_LOC,AMBITO,LATITUD,LONGITUD,LAT_DECIMAL,LON_DECIMAL,ALTITUD,CVE_CARTA,	
    POB_TOTAL,POB_MASCULINA,POB_FEMENINA,TOTAL_VIVIENDAS_HABITADAS) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

index = 1
for i, row in base.iterrows():
    #print(row)
    idx = index, 
    #print(f'valor i: {i}')
    cve_ent = row['CVE_ENT'],
    nom_ent = row['NOM_ENT'],
    nom_abr = row['NOM_ABR'],
    cve_mun = row['CVE_MUN'],
    nom_mun = row['NOM_MUN'],
    cve_loc = row['CVE_LOC'],
    nom_loc = row['NOM_LOC'],
    ambito = row['AMBITO'],
    lat = row['LATITUD'],
    lon = row['LONGITUD'],
    lat_decimal = row['LAT_DECIMAL'],
    lon_decimal = row['LON_DECIMAL'],
    altitud = row['ALTITUD'],
    cve_carta = row['CVE_CARTA'],	
    pob_total = row['POB_TOTAL'],
    pob_masculina = row['POB_MASCULINA'],
    pob_femenina = row['POB_FEMENINA'],
    total_viviendas = row['TOTAL.DE.VIVIENDAS.HABITADAS']
    #print(f'{idx[0]}')
    
    values = (idx[0],cve_ent[0],nom_ent[0],nom_abr[0],cve_mun[0],nom_mun[0],cve_loc[0],nom_loc[0],
              ambito[0],lat[0],lon[0],lat_decimal[0],lon_decimal[0],altitud[0],cve_carta[0],
              pob_total[0],pob_masculina[0],pob_femenina[0],total_viviendas[0])
    #print(values)
    cursor.execute(query.encode("utf-8"), values)
    index += 1
    #print(idx)
cursor.close()

db.commit()

Finalmente, realizamos una consulta para corroborar que se hayan cargado los datos, la otra opción es hacer la consulta directamente en la interfaz de MySQL.

In [26]:
# coneccion
db= mysql.connector.connect(user=user, 
                            password=password,
                            host=host,
                            database=database)

In [28]:
consulta = pd.read_sql('SELECT * from inegi;', con=db)
consulta.head()

c:\Users\jcmartinez\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,CVE_ENT,NOM_ENT,NOM_ABR,CVE_MUN,NOM_MUN,CVE_LOC,NOM_LOC,AMBITO,LATITUD,LONGITUD,LAT_DECIMAL,LON_DECIMAL,ALTITUD,CVE_CARTA,POB_TOTAL,POB_MASCULINA,POB_FEMENINA,TOTAL_VIVIENDAS_HABITADAS
0,1,1.0,Aguascalientes,Ags.,1.0,Aguascalientes,1.0,Aguascalientes,U,"21°52´47.362N""","102°17´45.768W""",22.0,-102.0,1878.0,F13D19,863893,419168,444725,2
1,2,1.0,Aguascalientes,Ags.,1.0,Aguascalientes,94.0,Granja Adelita,R,"21°52´18.749N""","102°22´24.710W""",22.0,-102.0,1901.0,F13D18,5,nan,nan,2
2,3,1.0,Aguascalientes,Ags.,1.0,Aguascalientes,96.0,Agua Azul,R,"21°53´01.522N""","102°21´25.639W""",22.0,-102.0,1861.0,F13D18,41,24,17,1
3,4,1.0,Aguascalientes,Ags.,1.0,Aguascalientes,100.0,Rancho Alegre,R,"21°51´16.556N""","102°22´21.884W""",22.0,-102.0,1879.0,F13D18,0,0,0,0
4,5,1.0,Aguascalientes,Ags.,1.0,Aguascalientes,102.0,Los Arbolitos [Rancho],R,"21°46´48.650N""","102°21´26.261W""",22.0,-102.0,1861.0,F13D18,8,nan,nan,2
